# Deep Learning - RNN Sentiment Analysis
---

## Author : Amir Atapour-Abarghouei, amir.atapour-abarghouei@durham.ac.uk

This notebook will provide an example that shows the implementation of a simple RNN for sentiment analysis in PyTorch.

Copyright (c) 2023 Amir Atapour-Abarghouei, UK.

License : LGPL - http://www.gnu.org/licenses/lgpl.html

Since [TorchData](https://github.com/pytorch/data) depends on nightly builds of PyTorch, to avoid any versioning issues, we will not be using [TorchText](https://pytorch.org/text/stable/index.html), and we'll be performing the data processing manually, but outside the Google Colab environment, you can use TorchText, which is clear and much better and has removed a lot of unnecessary abstraction.


We are going take advantage of the power of RNNs to build a model that is capable of detecting sentiment (i.e., whether a sentence is positive or negative) using PyTorch. This will be done on movie reviews, using the [IMDb dataset](http://ai.stanford.edu/~amaas/data/sentiment/).

Let's start by importing what we need:

In [ ]:
import os
import io
import tarfile
import requests
import re
import string
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

print(f'torch version: {torch.__version__}')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device is {device}!")

Now, we are going to download and unpack the dataset directly from the source:

In [ ]:
# Download IMDb dataset
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
r = requests.get(url)
with open('aclImdb_v1.tar.gz', 'wb') as file:
    file.write(r.content)

# Extract the dataset
with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as tar:
    tar.extractall()

print('done!')

Now, we process the data and prepare them to be used. The following code block defines two functions, `preprocess_text` and `load_data`, to process text data. `preprocess_text` converts text to lowercase, removes punctuation, and splits it into words, while `load_data` reads text files from specified directories, applies preprocessing, and labels them as positive (1) or negative (0), creating a dataset for sentiment analysis.

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(f'[{string.punctuation}]', '', text)
    return text.split()

def load_data(path):
    data = []
    for label in ['pos', 'neg']:
        directory = f'{path}/{label}'
        for filename in os.listdir(directory):
            if filename.endswith('.txt'):
                with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                    text = file.read()
                    text = preprocess_text(text)
                    data.append((1 if label == 'pos' else 0, text))
    return data

train_data = load_data('aclImdb/train')
test_data = load_data('aclImdb/test')

print('done!')

Now, we can get the dataset, which is already built into TorchText. The following command will download the IMDB dataset:

In [ ]:
print(f'\nThere are {len(train_data)} data points in the training set!')
print(f'There are {len(test_data)} data points in the testing set!')

Let's check one of the data samples:

In [ ]:
print(' '.join(train_data[0][1]))
print(f'label: {train_data[0][0]}')

Now is the time to build a vocabulary. The following block of code creates a vocabulary mapping words to unique indices from our dataset, assigning special indices for unknown and padding tokens, and defines a function to convert text data into fixed-length sequences of these indices, padding shorter sequences and truncating longer ones.

In [ ]:
# Build vocabulary
word_counts = Counter()
for _, text in train_data:
    word_counts.update(text)

vocab = {word: i+2 for i, (word, _) in enumerate(word_counts.most_common())}
vocab['<unk>'] = 1
vocab['<pad>'] = 0

# Encode and pad data
def encode_and_pad(data, vocab, max_length=500):
    encoded_data = []
    labels = []
    for label, text in data:
        encoded_text = [vocab.get(word, vocab['<unk>']) for word in text][:max_length]
        padded_text = encoded_text + [vocab['<pad>']] * (max_length - len(encoded_text))
        encoded_data.append(padded_text)
        labels.append(label)
    return np.array(encoded_data), np.array(labels)

X_train, y_train = encode_and_pad(train_data, vocab)
X_test, y_test = encode_and_pad(test_data, vocab)

print('done!')

Note that we also have the `<unk>` and `<pad>` tokens.

Now we make a class to handle our dataset cleanly. The following block of code defines the `IMDBDataset` class, a custom dataset for handling our IMDb movie review data, and creates instances for training and testing data. It also sets up `DataLoader` objects for both datasets, which facilitate efficient batching, shuffling, and loading of the data during model training and evaluation.

In [ ]:
class IMDBDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return torch.tensor(self.texts[idx]), torch.tensor(self.labels[idx])

train_dataset = IMDBDataset(X_train, y_train)
test_dataset = IMDBDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

print('The data is ready!')

Now, we will create our optimiser and our network, a very simple architecture made up of an embedding layer, an RNN layer and a linear layer:

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)  # Added batch_first=True
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        # text: [batch size, seq len]
        embedded = self.embedding(text)  # embedded: [batch size, seq len, emb dim]
        output, hidden = self.rnn(embedded)  # output: [batch size, seq len, hidden dim], hidden: [1, batch size, hidden dim]
        # Use the hidden state from the last layer
        hidden = hidden[-1]  # hidden: [batch size, hidden dim]
        return self.fc(hidden)


# a few parameters to set for our model:
input_dim = len(vocab)
embed_dim = 100
hidden_dim = 256
output_dim = 1

model = RNN(input_dim, embed_dim, hidden_dim, output_dim)
model = model.to(device)

print(model)
print('Model has been created!')
print(f'Model has {sum(p.numel() for p in model.parameters() if p.requires_grad)} parameters!')

# create the optimiser:
optimiser = torch.optim.SGD(model.parameters(), lr=1e-3)

# loss function:
criterion = nn.BCEWithLogitsLoss()

print('done!')

In order to keep track of how our model performs we need to calculate the accuracy of our predictions. This helper function will help us computer accuracy:

In [ ]:
# Function to calculate accuracy:
def binary_accuracy(preds, y):

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

print('done!')

And now the main training loop:

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_acc = 0

    model.train()  # Set the model to training mode
    for texts, labels in train_loader:
        texts, labels = texts.to(device), labels.to(device)

        # Forward pass
        predictions = model(texts)

        # Ensure predictions are the right shape
        predictions = predictions.squeeze()  # Remove any singleton dimensions

        # Compute loss
        loss = criterion(predictions, labels.float())
        # Calculate accuracy
        acc = binary_accuracy(predictions, labels)

        # Backward pass and optimization
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        epoch_loss += loss.item()/len(train_loader)
        epoch_acc += acc.item()/len(train_loader)

    print(f'Epoch: {epoch+1:02} || Train Loss: {epoch_loss:.3f} - Train Acc: {epoch_acc*100:.2f}%')

Now that the training is complete, we should evaluate the model:

In [ ]:
eval_loss = 0
eval_acc = 0

# Evaluation loop (example)
model.eval()
with torch.no_grad():
    for batch in test_loader:
        texts, labels = batch
        texts, labels = texts.to(device), labels.to(device)

        predictions = model(texts)
        predictions = predictions.squeeze()  

        # calculate loss and accuracy:
        loss = criterion(predictions, labels.float())
        acc = binary_accuracy(predictions, labels)
        eval_loss += loss.item() / len(test_loader)
        eval_acc += acc.item() / len(test_loader)

print(f'Test Loss: {eval_loss:.3f} - Test Acc: {eval_acc*100:.2f}%')

Don't be surprised, the results are awful. There are lots of ways to improve what we have done here. Try to see if you can improve things.